In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#Similarity transformation on extracted faces to be implemented
# Not pretrained

In [83]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, 3, stride=2, padding = 1)
        self.conv2 = nn.Conv2d(64, 128, 3, stride=2, padding = 1)
        self.conv3 = nn.Conv2d(128, 256, 3, stride=2, padding = 1)
        self.conv4 = nn.Conv2d(256, 512, 3, stride=2, padding = 1)
        
        self.fc1 = nn.Linear(6 * 7 * 512, 512)
        self.fc2 = nn.Linear(512, 3)

    def forward(self, x):
        conv1_output = F.relu(self.conv1(x))
#         print(conv1_output.size())
        conv2_output = F.relu(self.conv2(conv1_output))
#         print(conv2_output.size())
        conv3_output = F.relu(self.conv3(conv2_output))
#         print(conv3_output.size())
        conv4_output = F.relu(self.conv4(conv3_output))
#         print(conv4_output.size())
        conv4_output_flattened = conv4_output.view(-1, self.num_flat_features(conv4_output))
#         print(conv4_output_flattened.size())
        fc1_output = F.relu(self.fc1(conv4_output_flattened))
        fc2_output = F.softmax(self.fc2(fc1_output))
        
        return fc2_output
    
    def num_flat_features(self, x):
#         print(x.size())
        size  = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
#         print(num_features)
        return num_features     

In [84]:
net = Net().cuda()

In [85]:
print(net)

Net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (fc1): Linear(in_features=21504, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=3, bias=True)
)


In [86]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

12
torch.Size([64, 3, 3, 3])


In [87]:
input = torch.randn(1, 3, 96, 112).cuda()
out = net(input)
print(out)

tensor([[ 0.3330,  0.3393,  0.3277]], device='cuda:0')


C:\Users\Aarush\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [89]:
net.zero_grad()
out.backward(torch.randn(1, 3).cuda())